In [2]:
from pathlib import Path
import json
import xml.etree.ElementTree as ET

In [4]:
base = Path('/Users/hiro/Downloads/')
lib_json_f = base / 'My Library.json'
lib_rdf_f = base / 'My Library.rdf'

In [5]:
lib_json = json.load(open(lib_json_f, 'r'))

In [6]:
lib_rdf = ET.parse(lib_rdf_f)

In [65]:
doc_tags = set([r'{http://purl.org/net/biblio#}Book', r'{http://purl.org/net/biblio#}Article', r'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description'])
coll_tag = r'{http://www.zotero.org/namespaces/export#}Collection'

title_tag = r'{http://purl.org/dc/elements/1.1/}title'
link_tag = r'{http://purl.org/rss/1.0/modules/link/}link'
resource_tag = r'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource'
attachment_tag = r'{http://www.zotero.org/namespaces/export#}Attachment'
type_tag = r'{http://purl.org/rss/1.0/modules/link/}type'

about_key = r"{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about"
resource_key = r"{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource"

root = lib_rdf.getroot()
item_id2node = {}

docs = []
colls = []

for child in root:
    if about_key in child.attrib:
        item_id = child.attrib[about_key]
        item_id2node[item_id] = child
        #print(child.tag, item_id)
        if child.tag in doc_tags:
            docs.append(child)
        elif child.tag == coll_tag:
            colls.append(child)
    else:
        print('[ERROR] `about` attribute not found for top-level item!')
        print(child.tag, child.attrib)

In [86]:
all_docs = []

for doc in docs:
    item_id = doc.attrib[about_key]
    title = doc.find(title_tag).text
    attachments = []
    links = doc.findall(link_tag)
    for link in links:
        link_id = link.attrib[resource_key]
        link_node = item_id2node[link_id]
        if link_node.tag != attachment_tag:
            continue
        rsrc = link_node.find(resource_tag)
        if rsrc is None:
            print('[ERROR] `resource` tag not found for attachment!')
            continue
        resource_path = rsrc.attrib[resource_key]
        resource_path = resource_path.removeprefix('attachments:')

        typ = link_node.find(type_tag)
        if typ is None:
            print('[ERROR] `type` tag not found for attachment!')
            continue
        resource_type = typ.text
        attachments.append({'path': resource_path, 'type': resource_type})
    all_docs.append({
        'id': item_id,
        'title': title,
        'attachments': attachments,
        'tags': set(),
    })

[ERROR] `resource` tag not found for attachment!
[ERROR] `resource` tag not found for attachment!
[ERROR] `resource` tag not found for attachment!
[ERROR] `resource` tag not found for attachment!


In [87]:
all_docs

[{'id': '#item_1',
  'title': 'カルボニルストレス代謝障害と統合失調症',
  'attachments': [{'path': 'Journal Article/新井 et al_2015_カルボニルストレス代謝障害と統合失調症.pdf',
    'type': 'application/pdf'}],
  'tags': set()},
 {'id': '#item_2',
  'title': '心理学におけるベイズ統計モデリング',
  'attachments': [{'path': 'Journal Article/裕士_2018_心理学におけるベイズ統計モデリング.pdf',
    'type': 'application/pdf'}],
  'tags': set()},
 {'id': 'http://www.nature.com/articles/s41598-018-38355-z',
  'title': 'Neurometabolic and functional connectivity basis of prosocial behavior in early adolescence',
  'attachments': [{'path': 'Journal Article/Okada et al_2019_Neurometabolic and functional connectivity basis of prosocial behavior in early.pdf',
    'type': 'application/pdf'}],
  'tags': set()},
 {'id': 'https://academic.oup.com/cercor/article/31/6/2812/6082824',
  'title': 'Pubertal Testosterone and the Structure of the Cerebral Cortex in Young Men',
  'attachments': [{'path': 'Journal Article/Liao et al_2021_Pubertal Testosterone and the Structure of the Cer

In [88]:
docid2doc = {}
for doc in all_docs:
    docid2doc[doc['id']] = doc

In [80]:
coll_dict = {}

hasPart_tag = r'{http://purl.org/dc/terms/}hasPart'

for coll in colls:
    coll_id = coll.attrib[about_key]
    title = coll.find(title_tag).text
    items = []
    children = []
    for part in coll.findall(hasPart_tag):
        part_id = part.attrib[resource_key]
        if part_id.startswith('#collection_'):
            children.append(part_id)
        elif part_id in docid2doc:
            items.append(part_id)
        else:
            print('[ERROR] unknown item id:', part_id)
    coll_dict[coll_id] = {
        'id': coll_id,
        'title': title,
        'items': items,
        'children': children,
    }

In [82]:
for coll in coll_dict.values():
    for child in coll['children']:
        if child not in coll_dict:
            print('[ERROR] unknown collection id:', child)
        coll_dict[child]['parent'] = coll['id']

In [84]:
def dfs(coll):
    if 'parent' in coll:
        prefix = coll_dict[coll['parent']]['full_name'] + '/'
    else:
        prefix = ''
    coll['full_name'] = prefix + coll['title']
    for child in coll['children']:
        dfs(coll_dict[child])

for coll in coll_dict.values():
    if 'parent' not in coll: # depth 0 nodes
        dfs(coll)


In [89]:
for coll in coll_dict.values():
    for item in coll['items']:
        components = coll['full_name'].split('/')
        for i in range(len(components)):
            docid2doc[item]['tags'].add('/'.join(components[:i+1]))

In [91]:
all_docs

[{'id': '#item_1',
  'title': 'カルボニルストレス代謝障害と統合失調症',
  'attachments': [{'path': 'Journal Article/新井 et al_2015_カルボニルストレス代謝障害と統合失調症.pdf',
    'type': 'application/pdf'}],
  'tags': {'Carbonyl Stress'}},
 {'id': '#item_2',
  'title': '心理学におけるベイズ統計モデリング',
  'attachments': [{'path': 'Journal Article/裕士_2018_心理学におけるベイズ統計モデリング.pdf',
    'type': 'application/pdf'}],
  'tags': {'Psychology'}},
 {'id': 'http://www.nature.com/articles/s41598-018-38355-z',
  'title': 'Neurometabolic and functional connectivity basis of prosocial behavior in early adolescence',
  'attachments': [{'path': 'Journal Article/Okada et al_2019_Neurometabolic and functional connectivity basis of prosocial behavior in early.pdf',
    'type': 'application/pdf'}],
  'tags': {'TTC'}},
 {'id': 'https://academic.oup.com/cercor/article/31/6/2812/6082824',
  'title': 'Pubertal Testosterone and the Structure of the Cerebral Cortex in Young Men',
  'attachments': [{'path': 'Journal Article/Liao et al_2021_Pubertal Testosterone and

In [92]:
def parse_csl_jsonlib_json

[{'id': 'XinJingKarubonirusutoresuDaiXieZhangHaitoTongHeShiDiaoZheng2015',
  'type': 'article-journal',
  'container-title': '日本生物学的精神医学会誌',
  'DOI': '10.11249/jsbpjjpp.26.1_27',
  'issue': '1',
  'journalAbbreviation': '日本生物学的精神医学会誌',
  'page': '27-33',
  'title': 'カルボニルストレス代謝障害と統合失調症',
  'volume': '26',
  'author': [{'family': '新井', 'given': '誠'},
   {'family': '小堀', 'given': '晶子'},
   {'family': '宮下', 'given': '光弘'},
   {'family': '鳥海', 'given': '和也'},
   {'family': '堀内', 'given': '泰江'},
   {'family': '畠山', 'given': '幸子'},
   {'family': '内田', 'given': '美樹'},
   {'family': '井上', 'given': '智子'},
   {'family': '糸川', 'given': '昌成'}],
  'issued': {'date-parts': [['2015']]}},
 {'id': 'YuShiXinLiXueniokerubeizuTongJimoderingu2018',
  'type': 'article-journal',
  'container-title': '心理学評論',
  'DOI': '10.24602/sjpr.61.1_22',
  'issue': '1',
  'page': '22-41',
  'title': '心理学におけるベイズ統計モデリング',
  'volume': '61',
  'author': [{'family': '裕士', 'given': '清水'}],
  'issued': {'date-parts': [['2018']]

In [3]:
docs = json.load(open('docs.json', 'r'))

In [33]:
p = docs[0]['attachments'][0]['path']
p

'Journal Article/新井 et al_2015_カルボニルストレス代謝障害と統合失調症.pdf'

In [34]:
import fitz
import unicodedata
from dehyphen import FlairScorer, text_to_format
import langdetect

scorer = FlairScorer(lang="en")

In [35]:
base = Path('/Users/hiro/Library/CloudStorage/OneDrive-Personal/ZotAttachments')

In [36]:
doc = fitz.open(base / p)

In [38]:
def to_plain(lst):
    res = []
    for l1 in lst:
        for l2 in l1:
            for word in l2:
                res.append(word)
    return ' '.join(res).replace('  ', ' ')

def dehyphen(t):
    fmt = text_to_format(t)
    fixed_hyphens = scorer.dehyphen(fmt)
    return to_plain(fixed_hyphens)

def remove_newline(t):
    lines = t.split('\n')
    for i in range(len(lines) - 1):
        if len(lines[i]) > 0 and lines[i][-1].isascii() and len(lines[i+1]) > 0 and lines[i+1][0].isascii():
            lines[i] += ' '
    return ''.join(lines).replace('  ', ' ')

lang = None
for i, page in enumerate(doc):
    t = page.get_text()
    if lang is None:
        lang = langdetect.detect(t)
    t = unicodedata.normalize("NFKC", t)
    if lang == 'en':
        t = dehyphen(t)
    elif lang == 'ja':
        t = remove_newline(t)
    print(i, t)

0 27日本生物学的精神医学会誌 26 巻 1 号はじめに 統合失調症研究における克服すべき点として異種性の回避が挙げられる。統合失調症の診断は臨床所見や経過観察に基づいており,発症の病因と病態を示すバイオマーカーに基づいた診断基準は未だ存在しない。したがって,統合失調症の早期診断,治療,予防法の創出のためには,異種性回避を念頭にしたバイオマーカーの同定とその分子基盤に基づいた病因,病態の理解が重要である。筆者らは,一部の統合失調症患者に見られるカルボニルストレス代謝障害の同定1,3)とカルボニルストレスを呈する症例の臨床経過と特徴2,16)を明らかにしてきた。さらに,治療法の開発に向けて,カルボニルストレス解毒作用をもつピリドキサミン(ビタミン B6 成分の一つ)を用いた医師主導治験を実施した。 勝田らの研究グループは,急性期症例でのカルボニルストレスマーカーの変動を確認し,筆者らと同様にビタミン B6 補充療法の有用性と臨床特徴,経過との関連を報告している12)。国外では,フランスのグループが,蛍光性を有する終末糖化産物(Advanced glycation end products,AGEs)を AGE Reader(Diagnoptics Technologies B.V.)を利用して測定を行い,AGEs 蓄積を統合失調症で報告している13)。統合失調症に特異的なカルボニルストレスマーカー(ペントシジン,ビタミン B6)は,症例の階層化と病因,病態研究のツールとなることが期待される。これまでの研究から,ペントシジン値は入院患者に比べて外来患者で有意に低下し,また,数年間の追跡研究においても,ペントシジン低下と臨床症状の改善に相関が認められている。 一般的なマーカーの定義は,通常の生物学的過程,病理学的過程,もしくは治療的介入に対する薬理学的応答の指標として,客観的に測定され評価される特性とされ(表 1),基礎研究の成果を臨床応用していくためにも統合失調症の真のバイオマーカー同定へ期待が寄せられている。本稿で紹介するカルボニルストレスマーカーは,ある特定の比較的均一な  Carbonyl stress dysfunction in a subpopulation of schizophrenia *公益財団法人東京都医学総合研究所 精神行動医学研究分野 統合失

In [32]:
langdetect.detect(t)

'en'